<a href="https://colab.research.google.com/github/Jaime44/KUNVENO/blob/main/icefall_yes_no_dataset_recipe_with_CPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yesno recipe in icefall

This notebook shows you how to setup the environment to use [icefall][icefall] for training and decoding.
It also describes how to use a per-trained model to decode waves.


We use the [yesno] dataset as an example.

[icefall]: https://github.com/k2-fsa/icefall
[yesno]: https://www.openslr.org/1/

## Environment setup

In [1]:
import sys
import os


# Comprueba si el código se está ejecutando en Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

path_absolute = ''
if IN_COLAB:
    print("El código se está ejecutando en Google Colab.")
    from google.colab import drive

    drive.mount('/content/drive')
    path_absolute = '/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/'

    # Cambia al directorio de tu carpeta en Google Drive
    os.chdir(path_absolute)

    # Lista los archivos y carpetas en el directorio actual
    contenido_carpeta = os.listdir(path_absolute)
    print("Contenido de la carpeta en Google Drive:")
    print(contenido_carpeta)
else:
    print("El código se está ejecutando en un entorno local.")
    path_absolute = 'C:/Users/jaime/OneDrive - Universidad de Málaga/Escritorio/UNIR/PRÁCITCAS EXT/KUNVENO/KUNVENO_DIGITAL/'


pat_dir_checkpoints = path_absolute+'checkpoints'

El código se está ejecutando en Google Colab.
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Contenido de la carpeta en Google Drive:
['Git_commnads.txt', '.git', 'checkpoints', 'icefall', '.gitmodules', '.gitignore', 'icefall_yes_no_dataset_recipe_with_CPU.ipynb', 'README.md', 'pruned_transducer_statelessx_with_GPU.ipynb']


In [2]:
!ls -la

total 147
drwx------ 2 root root  4096 Oct 11 17:24 checkpoints
drwx------ 2 root root  4096 Oct 11 16:57 .git
-rw------- 1 root root  2401 Oct  5 18:23 Git_commnads.txt
-rw------- 1 root root   439 Oct 25 18:36 .gitignore
-rw------- 1 root root    79 Oct 16 19:29 .gitmodules
drwx------ 4 root root  4096 Oct 11 18:52 icefall
-rw------- 1 root root 55078 Oct 26 19:02 icefall_yes_no_dataset_recipe_with_CPU.ipynb
-rw------- 1 root root 78557 Nov 14 18:13 pruned_transducer_statelessx_with_GPU.ipynb
-rw------- 1 root root    87 Nov  1 08:25 README.md


In [3]:
!pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


### Install PyTorch and torchaudio

In [4]:
import torch
print(torch.__version__)

2.0.1+cu117


Colab pre-installs PyTorch, so we don't need to install it here.

From https://pytorch.org/audio/main/installation.html#compatibility-matrix, we need to install torchaudio==2.0.2 as the current PyTorch version is 2.0.1

In [5]:
! pip install torchaudio==2.0.2

### Install k2

We are going to install k2 by following https://k2-fsa.github.io/k2/installation/from_wheels.html.


In [6]:
# ! pip install k2==1.24.3.dev20230718+cuda11.8.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html
! pip install k2==1.24.3.dev20230718+cuda11.7.torch2.0.1 -f https://k2-fsa.github.io/k2/cuda.html

Looking in links: https://k2-fsa.github.io/k2/cuda.html


Check that k2 was installed successfully:

In [7]:
! python3 -m k2.version


k2 version: 1.24.3
Build type: Release
Git SHA1: e400fa3b456faf8afe0ee5bfe572946b4921a3db
Git date: Sat Jul 15 04:21:50 2023
Cuda used to build k2: 11.7
cuDNN used to build k2: 
Python version used to build k2: 3.10
OS used to build k2: CentOS Linux release 7.9.2009 (Core)
CMake version: 3.26.4
GCC version: 9.3.1
CMAKE_CUDA_FLAGS:  -Wno-deprecated-gpu-targets   -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_35,code=sm_35  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_50,code=sm_50  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_60,code=sm_60  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_61,code=sm_61  -lineinfo --expt-extended-lambda -use_fast_math -Xptxas=-w  --expt-extended-lambda -gencode arch=compute_70,code=sm_70  -lineinfo --expt-extended-lambda -u

### Install lhotse
[lhotse][lhotse] is used for data preparation.

[lhotse]: https://github.com/lhotse-speech/lhotse

Normally, we would use `pip install lhotse`. However, the yesno recipe is added recently and has not been released to PyPI yet, so we install the latest unreleased version here.

In [8]:
! pip install lhotse
! pip install git+https://github.com/lhotse-speech/lhotse

  Cloning https://github.com/lhotse-speech/lhotse to /tmp/pip-req-build-z_xzpcwo
  Running command git clone --filter=blob:none --quiet https://github.com/lhotse-speech/lhotse /tmp/pip-req-build-z_xzpcwo
  Resolved https://github.com/lhotse-speech/lhotse to commit 706117511588bf736601e034dc052e0426cbb3b0
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


### Install icefall

[icefall][icefall] is a collection of Python scripts.
You don't need to install it. What you need to do is
to get its source code, install its dependencies, and
set the `PYTHONPATH` pointing to it.

[icefall]: https://github.com/k2-fsa/icefall

In [9]:
! pwd

/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL


Now install dependencies of `icefall`:

In [10]:
!pip install typing-extensions==3.10.0.2


  Using cached typing_extensions-3.10.0.2-py3-none-any.whl (26 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.8.0
    Uninstalling typing_extensions-4.8.0:
      Successfully uninstalled typing_extensions-4.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sqlalchemy 2.0.23 requires typing-extensions>=4.2.0, but you have typing-extensions 3.10.0.2 which is incompatible.
arviz 0.15.1 requires typing-extensions>=4.1.0, but you have typing-extensions 3.10.0.2 which is incompatible.
chex 0.1.7 requires typing-extensions>=4.2.0; python_version < "3.11", but you have typing-extensions 3.10.0.2 which is incompatible.
flax 0.7.5 requires typing-extensions>=4.2, but you have typing-extensions 3.10.0.2 which is incompatible.
ibis-framework 6.2.0 requires typing-extensions<5,>=4.3.0, but you have typing-extensions 3.10

In [11]:
! cd icefall && \
  pip install -r requirements.txt

  Using cached typing_extensions-4.8.0-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 3.10.0.2
    Uninstalling typing-extensions-3.10.0.2:
      Successfully uninstalled typing-extensions-3.10.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.8.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 2.0.1 which is incompatible.


## Data preparation

We have set up the environment. Now it is the time to prepare data for training and decoding.

As we just said, `icefall` is a collection of Python scripts and we have to set up the `PYTHONPATH` variable to use it. Remember that `icefall` was downloaded to
`/content/icefall`, so we use

```
export PYTHONPATH=/content/icefall:$PYTHONPATH
```

**HINT**: You can have several versions of `icefall` in your virtual environemnt. To switch to a specific version of `icefall`, just change the `PYTHONPATH` environment variable.

In [12]:
# To remove the following warning message
# 2023-07-27 05:03:07.156920: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
! pip uninstall -y tensorflow

In [13]:
# !sudo chmod -R 755 icefall/egs/librispeech/ASR/

In [14]:
!rm -r icefall/egs/librispeech/ASR/shared && \
  sudo ln -s ../../../icefall/shared/ icefall/egs/librispeech/ASR/shared


In [15]:
!rm -r icefall/icefall/transformer_lm/scaling.py && \
ls -la icefall/icefall/transformer_lm/


total 69
-rw------- 1 root root 22349 Oct 12 19:52 attention.py
-rw------- 1 root root  5497 Oct 12 19:52 compute_perplexity.py
-rw------- 1 root root    20 Oct 12 19:52 dataset.py
-rw------- 1 root root 11234 Oct 12 19:52 encoder.py
-rw------- 1 root root  4925 Oct 12 19:52 export.py
-rw------- 1 root root     0 Oct 12 19:52 __init__.py
-rw------- 1 root root  3396 Oct 12 19:52 model.py
drwx------ 2 root root  4096 Oct 12 19:56 __pycache__
-rw------- 1 root root 17243 Oct 12 19:52 train.py


In [16]:
!sudo ln -s ../../egs/librispeech/ASR/pruned_transducer_stateless2/scaling.py icefall/icefall/transformer_lm/scaling.py

In [17]:
# !sudo ln -s ../../../librispeech/ASR/local/prepare_lang_fst.py icefall/egs/yesno/ASR/local/prepare_lang_fst.py && \
# sudo chmod 755 icefall/egs/yesno/ASR/local/prepare_lang_fst.py

In [18]:
!apt-get update && apt-get install dos2unix

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
dos2unix is already the newest version (7.4.2-2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [19]:
# !dos2unix icefall/egs/yesno/ASR/prepare.sh
# !dos2unix icefall/icefall/shared/parse_options.sh

In [20]:
# !find icefall -type f -print0 | xargs -0 dos2unix

In [21]:
!sudo chmod 755 icefall/egs/librispeech/ASR/prepare.sh

In [22]:
! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/librispeech/ASR && \
  ./prepare.sh
  # rm -rf data && \

2023-11-14 18:14:45 (prepare.sh:63:main) dl_dir: /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/download
2023-11-14 18:14:45 (prepare.sh:66:main) Stage -1: Download LM
2023-11-14 18:14:45 (prepare.sh:75:main) Stage 0: Download data
2023-11-14 18:14:45 (prepare.sh:97:main) Stage 1: Prepare LibriSpeech manifest
2023-11-14 18:14:45 (prepare.sh:108:main) Stage 2: Prepare musan manifest
2023-11-14 18:17:25,530 WARNING [qa.py:118] There are 15 recordings that do not have any corresponding supervisions in the SupervisionSet.
2023-11-14 18:19:31 (prepare.sh:119:main) Stage 3: Compute fbank for librispeech
2023-11-14 18:19:45,511 INFO [compute_fbank_librispeech.py:159] {'bpe_model': None, 'dataset': None, 'perturb_speed': True}
Traceback (most recent call last):
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/./local/compute_fbank_librispeech.py", line 160, in <module>
    compute_fbank_libris

In [23]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd icefall/egs/librispeech/ASR && \
#   ./prepare.sh --stage 1

In [24]:
!sudo chmod 755 icefall/egs/librispeech/ASR/pruned_transducer_stateless4/train.py

In [25]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [26]:
# !rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless2/encoder_interface.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/conformer.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decoder.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/joiner.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/model.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/encoder_interface.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling_converter.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/beam_search.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decode_stream.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/lstmp.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/streaming_beam_search.py && \
# rm -r icefall/egs/librispeech/ASR/pruned_transducer_stateless4/test_model.py && \
# rm -r icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# rm -r icefall/egs/librispeech/ASR/transducer/asr_datamodule.py && \
# ln -s ../tdnn_lstm_ctc/asr_datamodule.py icefall/egs/librispeech/ASR/transducer/asr_datamodule.py && \
# ln -s ../transducer/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# ln -s ../pruned_transducer_stateless2/optim.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py && \
# ln -s ../pruned_transducer_stateless2/conformer.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/conformer.py && \
# ln -s ../pruned_transducer_stateless2/decoder.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decoder.py && \
# ln -s ../pruned_transducer_stateless2/joiner.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/joiner.py && \
# ln -s ../pruned_transducer_stateless2/model.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/model.py && \
# ln -s ../pruned_transducer_stateless2/asr_datamodule.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/asr_datamodule.py && \
# ln -s ../pruned_transducer_stateless2/encoder_interface.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/encoder_interface.py && \
# ln -s ../pruned_transducer_stateless2/scaling.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling.py && \
# ln -s ../pruned_transducer_stateless3/scaling.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/scaling_converter.py && \
# ln -s ../pruned_transducer_stateless2/beam_search.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/beam_search.py && \
# ln -s ../pruned_transducer_stateless/decode_stream.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/decode_stream.py && \
# ln -s ../pruned_transducer_stateless/test_model.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/test_model.py && \
# ln -s ../pruned_transducer_stateless3/lstmp.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/lstmp.py && \
# ln -s ../pruned_transducer_stateless2/streaming_beam_search.py icefall/egs/librispeech/ASR/pruned_transducer_stateless4/streaming_beam_search.py && \
# ln -s ../tdnn_lstm_ctc/asr_datamodule.py icefall/egs/aishell/ASR/pruned_transducer_stateless2/asr_datamodule.py && \
# ln -s ../transducer_stateless/encoder_interface.py icefall/egs/librispeech/ASR/pruned_transducer_stateless2/encoder_interface.py

In [27]:
!ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2

total 69
lrw------- 1 root root    34 Oct 31 17:56 asr_datamodule.py -> ../tdnn_lstm_ctc/asr_datamodule.py
-rw------- 1 root root    68 Oct 12 19:35 beam_search.py
-rw------- 1 root root    66 Oct 12 19:35 conformer.py
-rw------- 1 root root 17117 Oct 12 19:35 decode.py
-rw------- 1 root root    64 Oct 12 19:35 decoder.py
-rw------- 1 root root    74 Oct 12 19:35 encoder_interface.py
-rw------- 1 root root  6236 Oct 12 19:35 export.py
-rw------- 1 root root    63 Oct 12 19:35 joiner.py
-rw------- 1 root root    62 Oct 12 19:35 model.py
-rw------- 1 root root    62 Oct 12 19:36 optim.py
-rw------- 1 root root  9284 Oct 12 19:35 pretrained.py
-rw------- 1 root root    64 Oct 12 19:35 scaling.py
-rw------- 1 root root 32062 Oct 12 19:36 train.py


In [28]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2

total 332
lrw------- 1 root root     31 Oct 31 17:56 asr_datamodule.py -> ../transducer/asr_datamodule.py
-rwx------ 1 root root 100529 Oct 12 19:40 beam_search.py
-rwx------ 1 root root  64089 Oct 12 19:40 conformer.py
-rwx------ 1 root root  25947 Oct 12 19:41 decode.py
-rwx------ 1 root root   4425 Oct 12 19:40 decoder.py
-rwx------ 1 root root     47 Oct 12 19:40 decode_stream.py
-rwx------ 1 root root     44 Oct 31 17:56 encoder_interface.py
-rwx------ 1 root root   6780 Oct 12 19:40 export.py
-rwx------ 1 root root      0 Oct 12 19:40 __init__.py
-rwx------ 1 root root   2181 Oct 12 19:41 joiner.py
-rwx------ 1 root root   7445 Oct 12 19:41 model.py
-rwx------ 1 root root  11877 Oct 12 19:41 optim.py
-rwx------ 1 root root  11429 Oct 12 19:41 pretrained.py
-rwx------ 1 root root  36364 Oct 12 19:41 scaling.py
-rwx------ 1 root root   9492 Oct 12 19:41 streaming_beam_search.py
-rwx------ 1 root root  18386 Oct 12 19:41 streaming_decode.py
-rwx------ 1 root root     44 Oct 12 19:41

In [29]:
!ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4

total 122
lrw------- 1 root root    49 Oct 31 17:56 asr_datamodule.py -> ../pruned_transducer_stateless2/asr_datamodule.py
-rwx------ 1 root root    46 Oct 31 17:56 beam_search.py
-rwx------ 1 root root    44 Oct 31 17:56 conformer.py
-rwx------ 1 root root 32416 Oct 12 19:41 decode.py
-rwx------ 1 root root    42 Oct 31 17:56 decoder.py
-rwx------ 1 root root    47 Oct 31 17:56 decode_stream.py
-rwx------ 1 root root    52 Oct 31 17:56 encoder_interface.py
drwx------ 4 root root  4096 Oct 30 19:27 exp
-rwx------ 1 root root  9492 Oct 12 19:41 export.py
-rwx------ 1 root root    43 Oct 12 19:41 __init__.py
-rwx------ 1 root root    41 Oct 31 17:56 joiner.py
-rwx------ 1 root root    40 Oct 31 17:56 lstmp.py
-rwx------ 1 root root    40 Oct 31 17:56 model.py
-rwx------ 1 root root    40 Oct 31 17:56 optim.py
-rwx------ 1 root root  4030 Oct 12 19:41 profile.py
drwx------ 3 root root  4096 Oct 30 19:09 pruned_transducer_stateless4
drwx------ 2 root root  4096 Oct 29 13:24 __pycache__
-rw

In [30]:
# !export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
#   ls -la icefall/egs/librispeech/ASR/pruned_transducer_stateless2 && \
#   ls -la icefall/egs/librispeech/ASR/transducer && \
#   ls -la icefall/egs/aishell/ASR/pruned_transducer_stateless2 && \
#   cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
#   ./train.py --help

In [31]:
! nvidia-smi

Tue Nov 14 18:19:49 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [32]:
!export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  export CUDA_VISIBLE_DEVICES="0" && \
  ./train.py --world-size 1

Traceback (most recent call last):
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/pruned_transducer_stateless4/./train.py", line 68, in <module>
    import optim
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py", line 1
    ../pruned_transducer_stateless2/optim.py
    ^
SyntaxError: invalid syntax


In [33]:
!export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
  export CUDA_VISIBLE_DEVICES="0" && \
  cd icefall/egs/librispeech/ASR/pruned_transducer_stateless4 && \
  ./train.py \
   --world-size 4 \
   --dynamic-chunk-training 1 \
   --causal-convolution 1 \
   --num-epochs 30 \
   --start-epoch 1 \
   --exp-dir pruned_transducer_stateless4/exp \
   --full-libri 1 \
   --max-duration 300

Traceback (most recent call last):
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/pruned_transducer_stateless4/./train.py", line 68, in <module>
    import optim
  File "/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/librispeech/ASR/pruned_transducer_stateless4/optim.py", line 1
    ../pruned_transducer_stateless2/optim.py
    ^
SyntaxError: invalid syntax


------------------------------------------------------------------------------------------------------------------------------------------------------------

In [34]:
!echo $CUDA_VISIBLE_DEVICES


-----------------------------------------------------------------------------------------------------------------------------------------------------------

## Training

In [35]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
# cd icefall/egs/yesno/ASR && \
# ./tdnn/train.py

In [36]:
# !ls -la


In [37]:
# !cd icefall/egs/yesno/ASR/tdnn/exp/tensorboard && \
# tensorboard dev upload --logdir . --description "TDNN training for yesno with icefall"

## Decoding

In [38]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/decode.py

### Show the decoding result

In [39]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/recogs-test_set.txt

### Show the detailed WER

In [40]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cat tdnn/exp/errs-test_set.txt

# Pre-trained model

### Download the pre-trained model

In [41]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir tmp && \
# #   cd tmp && \
# #   git lfs install && \
# #   git clone https://huggingface.co/csukuangfj/icefall_asr_yesno_tdnn
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   cd tmp &&

In [42]:
# #! sudo apt-get install git-lfs

In [43]:
# ! sudo apt-get install tree

In [44]:
# # ! cd /content/icefall/egs/yesno/ASR && \
# #   mkdir -p tmp && \
# #   tree tmp

# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   tree tmp

In [45]:
# ! sudo apt-get install sox

In [46]:
# ! cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   soxi tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

## Download kaldifeat

See https://csukuangfj.github.io/kaldifeat/installation/from_wheels.html

In [47]:
# ! pip install kaldifeat==1.25.0.dev20230726+cuda11.8.torch2.0.1  -f https://csukuangfj.github.io/kaldifeat/cuda.html

## Inference with a pre-trained model


### View help information

In [48]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py --help

### Decode a single sound file

In [49]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav

### Decode multiple sound files

In [50]:
# ! export PYTHONPATH=/content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall:$PYTHONPATH && \
#   cd /content/drive/Othercomputers/Mi_portátil/KUNVENO/KUNVENO_DIGITAL/icefall/egs/yesno/ASR && \
#   ./tdnn/pretrained.py \
#     --checkpoint ./tmp/icefall_asr_yesno_tdnn/pretrained.pt \
#     --words-file ./tmp/icefall_asr_yesno_tdnn/lang_phone/words.txt \
#     --HLG ./tmp/icefall_asr_yesno_tdnn/lang_phone/HLG.pt \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/0_0_1_0_1_0_0_1.wav \
#     ./tmp/icefall_asr_yesno_tdnn/test_waves/1_0_1_1_0_1_1_1.wav